# HELP - CDFS-SWIRE Master List Creation

This notebook presents the creation of the HELP master catalogue on the CDFS-SWIRE field. The following table summarises the list of used catalogues.

| Survey    | Telescope / Instrument      |      Filters (detection band in bold)      | Location                    |
|-----------|-----------------------------|:------------------------------------------:|-----------------------------|
| VIDEO     | VISTA/VIRCAM                | **u**,**g**,**r**,**i**,**z** |dmu0_VISTA-VIDEO, dmu0_VISTA-VIDEO-private|
| SWIRE     | Spitzer / IRAC              | **IRAC1**,**IRAC2**,**IRAC3**,**IRAC4**    | dmu0_DataFusion-Spitzer     |
| SERVS     | Spitzer / IRAC              | **IRAC1**, **IRAC2**                       | dmu0_DataFusion-Spitzer     |
| PS1 3PSS  | Pan-STARRS1 / Pan-STARRS1   | grizy                                      | dmu0_PanSTARRS1-3SS         |
| PS1 MDS	| Pan-STARRS1 / Pan-STARRS1   |	grizy                                      | ...awaiting release         |
| VOICE	    | VST/OmegaCAM	              | u,g,r,i                                    | ...waiting for Mattia       |
| CTIO/CDFS | CTIO/MOSAIC	              | g,r,i                                      | ...waiting for Mattia       |
| DES-DEEP	| Blanco/DECAM	              | grizy                                      | ...awaiting release         |
| Fireworks	| HST	ACS	                  | U38, B435, B, V, V606, R, i775, I, z850, J, H, **Ks**, IRAC1, IRAC2, IRAC3, IRAC4   | dmu0_Fireworks              |
| COMBO-17	| ESO/MPG 	/WFI	          |                                            | dmu0_COMBO-17               |
| GOODS	    | Spitzer		              |                                            | ...unknown status           |


In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
c40aaa9 (Wed May 31 17:58:17 2017 +0100) [with local modifications]


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from astropy.table import Column, Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib as mpl
import numpy as np
import seaborn as sns
from pymoc import MOC
from matplotlib_venn import venn3

from herschelhelp_internal import flagging, utils, masterlist, starmask

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
import locale
locale.setlocale(locale.LC_ALL, 'en_GB')

'en_GB'

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## I - Pristine catalogues preparation

### I.a -VIDEO/VISTA/VIRCAM

The catalogue comes from `dmu0_VISTA_VIDEO-private`.

There is an old public version of the catalogue but we are using the newer private version in the hope that it will be public by the time we publish the masterlist.

Filters: Z,Y,J,H,K 

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position (degrees);
- The stellarity;
- The magnitude for each band in apertude 3, which is 2 arcsec (rs548 presumes same for private catalogue).
- The “auto” magnitude is provided, we presume this is standard Sextractor units etc.

Yannick said the dates of observation for VIDEO are from 2009/11 to 2016/12. There is a paper from 2012 (Jarvis et al). So will use 2012.

In [6]:
#Both flux and mag available in pristine catalogues
#I think these are still 2 arcsec aperture phtometriers and need to be replaceed with 3 arcsec.
#There is a PSTAR for each band and they vary quite a bit. I took the J band because it is in the middle
video = Table.read("../../dmu0/dmu0_VISTA-VIDEO-private/data/VIDEO-all_2016-04-14_fullcat_errfix_CDFS-SWIRE.fits")[
    'ID', 'ALPHA_J2000', 'DELTA_J2000', 'J_CLASS_STAR',     
    'Z_MAG_APER_3', 'Z_MAGERR_APER_3', 'Z_MAG_AUTO', 'Z_MAGERR_AUTO',
    'Z_FLUX_APER_3', 'Z_FLUXERR_APER_3', 'Z_FLUX_AUTO', 'Z_FLUXERR_AUTO',
    'Y_MAG_APER_3', 'Y_MAGERR_APER_3', 'Y_MAG_AUTO', 'Y_MAGERR_AUTO',
    'Y_FLUX_APER_3', 'Y_FLUXERR_APER_3', 'Y_FLUX_AUTO', 'Y_FLUXERR_AUTO',
    'J_MAG_APER_3', 'J_MAGERR_APER_3', 'J_MAG_AUTO', 'J_MAGERR_AUTO',
    'J_FLUX_APER_3', 'J_FLUXERR_APER_3', 'J_FLUX_AUTO', 'J_FLUXERR_AUTO',
    'H_MAG_APER_3', 'H_MAGERR_APER_3', 'H_MAG_AUTO', 'H_MAGERR_AUTO',
    'H_FLUX_APER_3', 'H_FLUXERR_APER_3', 'H_FLUX_AUTO', 'H_FLUXERR_AUTO',
    'K_MAG_APER_3', 'K_MAGERR_APER_3', 'K_MAG_AUTO', 'K_MAGERR_AUTO',
    'K_FLUX_APER_3', 'K_FLUXERR_APER_3', 'K_FLUX_AUTO', 'K_FLUXERR_AUTO']
video = Table(video.as_array(), names=[
    'video_id', 'video_ra', 'video_dec', 'video_stellarity',
    'm_app_video_z', 'merr_app_video_z', 'm_video_z', 'merr_video_z',
    'f_app_video_z', 'ferr_app_video_z', 'f_video_z', 'ferr_video_z',
    'm_app_video_y', 'merr_app_video_y', 'm_video_y', 'merr_video_y',
    'f_app_video_y', 'ferr_app_video_y', 'f_video_y', 'ferr_video_y',
    'm_app_video_j', 'merr_app_video_j', 'm_video_j', 'merr_video_j',
    'f_app_video_j', 'ferr_app_video_j', 'f_video_j', 'ferr_video_j',
    'm_app_video_h', 'merr_app_video_h', 'm_video_h', 'merr_video_h',
    'f_app_video_h', 'ferr_app_video_h', 'f_video_h', 'ferr_video_h',
    'm_app_video_k', 'merr_app_video_k', 'm_video_k', 'merr_video_k',
    'f_app_video_k', 'ferr_app_video_k', 'f_video_k', 'ferr_video_k',
])

video_epoch = 2012
#MOC made using stilts: stilts pixfoot in=VIDEO-all_2016-04-14_fullcat_errfix_ELAIS-S1.fits
#out=VIDEO-all_2016-04-14_fullcat_errfix_ELAIS-S1_MOC.fits ra=ALPHA_J2000 dec=DELTA_J2000 order=13
video_moc = MOC(filename="../../dmu0/dmu0_VISTA-VIDEO-private/data/VIDEO-all_2016-04-14_fullcat_errfix_CDFS-SWIRE_MOC.fits")

# Adding band-flag columns
for col in video.colnames:
    if col.startswith('m_'):
        
        #errcol = "merr{}".format(col[1:])
        #flux, error = utils.mag_to_flux(np.array(video[col]), np.array(video[errcol]))

        # Fluxes are added in µJy
        #video.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        #video.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        video.add_column(Column(np.zeros(len(video), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [7]:
video[:10].show_in_notebook()

idx,video_id,video_ra,video_dec,video_stellarity,m_app_video_z,merr_app_video_z,m_video_z,merr_video_z,f_app_video_z,ferr_app_video_z,f_video_z,ferr_video_z,m_app_video_y,merr_app_video_y,m_video_y,merr_video_y,f_app_video_y,ferr_app_video_y,f_video_y,ferr_video_y,m_app_video_j,merr_app_video_j,m_video_j,merr_video_j,f_app_video_j,ferr_app_video_j,f_video_j,ferr_video_j,m_app_video_h,merr_app_video_h,m_video_h,merr_video_h,f_app_video_h,ferr_app_video_h,f_video_h,ferr_video_h,m_app_video_k,merr_app_video_k,m_video_k,merr_video_k,f_app_video_k,ferr_app_video_k,f_video_k,ferr_video_k,flag_app_video_z,flag_video_z,flag_app_video_y,flag_video_y,flag_app_video_j,flag_video_j,flag_app_video_h,flag_video_h,flag_app_video_k,flag_video_k
0,1307519,52.5764425036,-28.0036165902,0.0103968,23.1244,0.0856302,22.9351,0.138891,562.637,44.3758,669.844,85.6917,22.8123,0.087532,22.6132,0.145146,750.06,60.4718,900.997,120.453,22.5619,0.0792798,22.4394,0.137883,944.59,68.9757,1057.4,134.289,22.2869,0.0997219,22.1966,0.17553,1216.91,111.774,1322.44,213.805,21.6483,0.0821624,21.5571,0.143083,2191.18,165.822,2383.2,314.079,False,False,False,False,False,False,False,False,False,False
1,1307537,52.5603041414,-28.0034513965,0.0729567,23.9185,0.177761,23.9029,0.336246,270.763,44.332,274.681,85.07,23.5323,0.169923,23.5279,0.33662,386.446,60.4827,388.002,120.299,23.5371,0.194704,23.4868,0.360703,384.726,68.995,402.968,133.878,23.2214,0.235867,23.1842,0.434417,514.565,111.789,532.494,213.065,22.9315,0.267905,22.9023,0.492546,672.046,165.832,690.365,313.196,False,False,False,False,False,False,False,False,False,False
2,1307545,52.56806518,-28.0034720553,0.0404727,21.4484,0.0182901,21.2843,0.0305287,2634.17,44.3761,3064.05,86.1577,21.2754,0.0212568,21.1132,0.0365021,3089.02,60.4795,3586.77,120.59,21.0487,0.0196787,20.9174,0.0340309,3806.45,68.9933,4295.74,134.649,20.828,0.0260207,20.752,0.0465374,4664.45,111.792,5002.51,214.427,20.6027,0.0313811,20.4955,0.053958,5740.28,165.917,6336.06,314.895,False,False,False,False,False,False,False,False,False,False
3,1307547,52.6574343795,-28.0038182732,0.00168766,26.6988,2.30199,26.9262,5.41295,20.9152,44.3462,16.9637,84.5755,24.5844,0.447751,24.6243,0.922916,146.635,60.4735,141.349,120.156,24.1176,0.332201,24.2266,0.710945,225.41,68.9706,203.871,133.5,22.8995,0.175361,22.9562,0.351068,692.145,111.795,656.931,212.423,22.6307,0.203072,22.6578,0.392258,886.622,165.836,864.739,312.426,False,False,False,False,False,False,False,False,False,False
4,1307553,52.7597187859,-28.0042276187,0.762026,23.2616,0.0954313,23.4266,0.214152,495.872,43.5863,425.969,84.0214,23.0878,0.112845,23.2196,0.252911,581.936,60.4851,515.424,120.067,23.1501,0.136275,23.1841,0.271659,549.468,68.9681,532.567,133.257,22.7953,0.158786,22.8781,0.32586,761.873,111.426,705.945,211.881,22.956,0.274104,22.9585,0.5166,657.057,165.885,655.539,311.92,False,False,False,False,False,False,False,False,False,False
5,1307559,52.6922441369,-28.0039817321,0.981548,21.4363,0.0180672,21.4903,0.0361918,2663.65,44.3259,2534.33,84.4819,21.0571,0.0173892,21.1005,0.0359413,3777.21,60.498,3629.2,120.142,20.8283,0.016076,20.8697,0.0322816,4663.03,69.0455,4488.85,133.469,20.8261,0.0259885,20.854,0.0506186,4672.75,111.852,4554.28,212.334,20.9607,0.0436281,20.9933,0.0846454,4127.81,165.873,4005.8,312.308,False,False,False,False,False,False,False,False,False,False
6,1307566,52.5415573606,-28.0032513622,0.00021876,23.2598,0.0957977,23.0407,0.153296,496.683,43.8253,607.726,85.8083,23.3198,0.139642,23.2777,0.26807,469.998,60.4509,488.565,120.631,22.8386,0.102236,23.0488,0.242567,732.049,68.9342,603.21,134.769,22.6178,0.134811,22.4296,0.218087,897.213,111.406,1067.0,214.33,22.4988,0.179821,22.5501,0.358374,1001.07,165.804,954.879,315.192,False,False,False,False,False,False,False,False,False,False
7,1307567,52.7803303559,-28.0042248642,0.0792431,25.1813,0.559335,26.1469,2.62526,84.6218,43.5958,34.7742,84.0851,24.8803,0.587909,24.9757,1.27495,111.659,60.4635,102.259,120.084,24.4588,0.454699,2

### I.b - Spitzer datafusion SERVS

The Spitzer catalogues were produced by the datafusion team are available in the HELP virtual observatory server. They are described there: http://vohedamtest.lam.fr/browse/df_spitzer/q.

Lucia told that the magnitudes are aperture corrected.

In the catalouge, we keep:

- The internal identifier (this one is only in HeDaM data);
- The position;
- The fluxes in aperture 2 (1.9 arcsec);
- The “auto” flux (which seems to be the Kron flux);
- The stellarity in each band

A query of the position in the Spitzer heritage archive show that the SERVS-ELAIS-N1 images were observed in 2009. Let's take this as epoch.

In [9]:
servs = Table.read("../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SERVS_CDFS-SWIRE.fits")[
    'internal_id', 'ra_12', 'dec_12',   
    'flux_aper_2_1', 'fluxerr_aper_2_1', 'flux_auto_1', 'fluxerr_auto_1', 'class_star_1',
    'flux_aper_2_2', 'fluxerr_aper_2_2', 'flux_auto_2', 'fluxerr_auto_2', 'class_star_2']

servs = Table(servs.as_array(), names=[
    'servs_intid', 'servs_ra', 'servs_dec',
    'f_app_servs_irac1', 'ferr_app_servs_irac1', 'f_servs_irac1', 'ferr_servs_irac1', 'servs_stellarity_irac1',
    'f_app_servs_irac2', 'ferr_app_servs_irac2', 'f_servs_irac2', 'ferr_servs_irac2', 'servs_stellarity_irac2'
])
servs_epoch = 2009
servs_moc = MOC(filename="../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SERVS_CDFS-SWIRE_MOC.fits")

# Adding magnitude and band-flag columns
for col in servs.colnames:
    if col.startswith('f_'):
        errcol = "ferr{}".format(col[1:])
        
        magnitude, error = utils.flux_to_mag(
            np.array(servs[col])/1.e6, np.array(servs[errcol])/1.e6)
        # Note that some fluxes are 0.
        
        servs.add_column(Column(magnitude, name="m{}".format(col[1:])))
        servs.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        # Band-flag column
        servs.add_column(Column(np.zeros(len(servs), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

/Users/rs548/GitHub/herschelhelp_internal/herschelhelp_internal/utils.py:76: RuntimeWarning: invalid value encountered in log10
  magnitudes = 2.5 * (23 - np.log10(fluxes)) - 48.6


In [10]:
servs[:10].show_in_notebook()

idx,servs_intid,servs_ra,servs_dec,f_app_servs_irac1,ferr_app_servs_irac1,f_servs_irac1,ferr_servs_irac1,servs_stellarity_irac1,f_app_servs_irac2,ferr_app_servs_irac2,f_servs_irac2,ferr_servs_irac2,servs_stellarity_irac2,m_app_servs_irac1,merr_app_servs_irac1,flag_app_servs_irac1,m_servs_irac1,merr_servs_irac1,flag_servs_irac1,m_app_servs_irac2,merr_app_servs_irac2,flag_app_servs_irac2,m_servs_irac2,merr_servs_irac2,flag_servs_irac2
0,763670,52.5304604,-27.9983577,nan,nan,nan,nan,nan,-0.134956646213,0.484765812009,-0.343099350045,0.441702320138,0.03,nan,nan,False,nan,nan,False,nan,-3.89997682735,False,nan,-1.39776481837,False
1,763797,52.5314016,-27.9972923,nan,nan,nan,nan,nan,0.51079663679,0.514715193284,-0.364548270618,0.854743044328,0.01,nan,nan,False,nan,nan,False,24.6293799273,1.09406538774,False,nan,-2.54568611015,False
2,424913,52.5289081,-27.9931454,4.03532899085,0.375223398818,2.65236576715,0.593315167577,0.31,4.27969028198,0.483283984068,3.95912366056,0.546072840276,0.67,22.3853026312,0.100956732373,False,22.8409164649,0.242871389101,False,22.3214691486,0.122606750515,False,22.4060023328,0.149753102948,False
3,763878,52.530666,-27.9948181,nan,nan,nan,nan,nan,9.3956923803,0.488043472599,7.37509960684,0.561865778001,0.55,nan,nan,False,nan,nan,False,21.4676780267,0.0563967450453,False,21.7305802745,0.0827159021452,False
4,424063,52.53300045,-27.99605695,2.07879361106,0.393492846997,1.23500891797,1.16624229009,0.05,2.00907092872,0.479472184446,1.51239916477,0.94836417509,0.09,23.1054715665,0.205517963893,False,23.6708247659,1.02528124247,False,23.1425118264,0.259114948301,False,23.4508339279,0.680821137818,False
5,763451,52.5337783,-27.9962791,nan,nan,nan,nan,nan,0.89352664257,0.484976488361,0.779084740156,0.54101862617,0.15,nan,nan,False,nan,nan,False,24.0222312337,0.589301434096,False,24.1710382551,0.75396613437,False
6,763376,52.5349252,-27.996121,nan,nan,nan,nan,nan,0.485477051589,0.478385936232,0.45539647404,0.414164691747,0.49,nan,nan,False,nan,nan,False,24.6845782357,1.06987741051,False,24.7540258407,0.987433206439,False
7,423891,52.5332938,-27.9931574,0.930696067594,0.402707362708,0.310153068172,0.467980197875,0.02,0.69938862107,0.420433388104,0.42389964919,0.367812838535,0.26,23.9779803027,0.469792426162,False,25.1710597956,1.63823316963,False,24.2882035947,0.652683983413,False,24.8318423571,0.942080787602,False
8,763475,52.5339033,-27.9940559,nan,nan,nan,nan,nan,0.0758123461801,0.428785724178,0.244294915666,0.187243615906,0.2,nan,nan,False,nan,nan,False,26.7006501574,6.14079643067,False,25.430213929,0.832179304039,False
9,763268,52.5353224,-27.9985832,nan,nan,nan,nan,nan,0.0455483927188,0.482642260591,-0.00235812422749,0.438264222059,0.04,nan,nan,False,nan,nan,False,27.2538173588,11.5047347445,False,nan,-201.787220364,False


### I.c - Spitzer datafusion SWIRE

The Spitzer catalogues were produced by the datafusion team are available in the HELP virtual observatory server. They are described there: http://vohedamtest.lam.fr/browse/df_spitzer/q.

Lucia told that the magnitudes are aperture corrected.

In the catalouge, we keep:

We keep:
- The internal identifier (this one is only in HeDaM data);
- The position;
- The fluxes in aperture 2 (1.9 arcsec) for IRAC bands.
- The Kron flux;
- The stellarity in each band

A query of the position in the Spitzer heritage archive show that the ELAIS-N1 images were observed in 2004. Let's take this as epoch.

We do not use the MIPS fluxes as they will be extracted on MIPS maps using XID+.

In [11]:
swire = Table.read("../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SWIRE_CDFS-SWIRE.fits")[
    'internal_id', 'ra_spitzer', 'dec_spitzer',      
    'flux_ap2_36', 'uncf_ap2_36', 'flux_kr_36', 'uncf_kr_36', 'stell_36',
    'flux_ap2_45', 'uncf_ap2_45', 'flux_kr_45', 'uncf_kr_45', 'stell_45',
    'flux_ap2_58', 'uncf_ap2_58', 'flux_kr_58', 'uncf_kr_58', 'stell_58',
    'flux_ap2_80', 'uncf_ap2_80', 'flux_kr_80', 'uncf_kr_80', 'stell_80']

swire = Table(swire.as_array(), names=[
    'swire_intid', 'swire_ra', 'swire_dec',
    'f_app_swire_irac1', 'ferr_app_swire_irac1', 'f_swire_irac1', 'ferr_swire_irac1', 'swire_stellarity_irac1',
    'f_app_swire_irac2', 'ferr_app_swire_irac2', 'f_swire_irac2', 'ferr_swire_irac2', 'swire_stellarity_irac2',
    'f_app_irac3', 'ferr_app_irac3', 'f_irac3', 'ferr_irac3', 'swire_stellarity_irac3',
    'f_app_irac4', 'ferr_app_irac4', 'f_irac4', 'ferr_irac4', 'swire_stellarity_irac4'
])
swire_epoch = 2004
swire_moc = MOC(filename="../../dmu0/dmu0_DataFusion-Spitzer/data/DF-SWIRE_CDFS-SWIRE_MOC.fits")
        
# Adding magnitude and band-flag columns
for col in swire.colnames:
    if col.startswith('f_'):
        errcol = "ferr{}".format(col[1:])
        
        magnitude, error = utils.flux_to_mag(
            np.array(swire[col])/1.e6, np.array(swire[errcol])/1.e6)
        # Note that some fluxes are 0.
        
        swire.add_column(Column(magnitude, name="m{}".format(col[1:])))
        swire.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        # Band-flag column
        swire.add_column(Column(
                np.zeros(len(swire), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [12]:
swire[:10].show_in_notebook()

idx,swire_intid,swire_ra,swire_dec,f_app_swire_irac1,ferr_app_swire_irac1,f_swire_irac1,ferr_swire_irac1,swire_stellarity_irac1,f_app_swire_irac2,ferr_app_swire_irac2,f_swire_irac2,ferr_swire_irac2,swire_stellarity_irac2,f_app_irac3,ferr_app_irac3,f_irac3,ferr_irac3,swire_stellarity_irac3,f_app_irac4,ferr_app_irac4,f_irac4,ferr_irac4,swire_stellarity_irac4,m_app_swire_irac1,merr_app_swire_irac1,flag_app_swire_irac1,m_swire_irac1,merr_swire_irac1,flag_swire_irac1,m_app_swire_irac2,merr_app_swire_irac2,flag_app_swire_irac2,m_swire_irac2,merr_swire_irac2,flag_swire_irac2,m_app_irac3,merr_app_irac3,flag_app_irac3,m_irac3,merr_irac3,flag_irac3,m_app_irac4,merr_app_irac4,flag_app_irac4,m_irac4,merr_irac4,flag_irac4
0,300465,52.528312,-28.00234,9.69,0.7,13.23,1.11,0.43,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,21.4341905574,0.0784329559681,False,21.0961003895,0.0910935137779,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
1,300519,52.532072,-28.00108,nan,nan,nan,nan,nan,25.92,1.05,28.06,1.38,0.31,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,False,20.365912507,0.0439823694057,False,20.2797808333,0.0533968625291,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
2,300425,52.539842,-28.00315,29.76,0.88,31.36,1.15,0.66,21.13,1.01,18.29,1.07,0.71,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,20.2159176828,0.0321051028289,False,20.159059865,0.0398149437332,False,20.5877512574,0.0518974712166,False,20.7444657363,0.0635176456583,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
3,300422,52.541612,-28.00324,8.36,0.64,15.37,1.12,0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,21.5944843064,0.0831185611298,False,20.9333153313,0.0791167566252,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
4,300589,52.531762,-27.99931,nan,nan,nan,nan,nan,23.4,1.01,17.95,0.81,0.82,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,False,20.4769603565,0.0468629729404,False,20.7648388677,0.0489942242816,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
5,300799,52.530612,-27.99477,10.43,0.64,9.12,0.7,0.51,8.82,0.88,7.0,0.93,0.07,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,21.3542892289,0.0666223558049,False,21.5000129042,0.0833350157161,False,21.5363285372,0.10832742179,False,21.7872549,0.144247810061,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
6,300534,52.537432,-28.00087,15.85,0.75,11.8,1.01,0.27,13.86,0.92,11.4,0.92,0.33,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,20.8999268336,0.0513755301936,False,21.2202949817,0.0929316582039,False,21.0455919243,0.0720690698685,False,21.2577378717,0.0876208165243,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
7,300636,52.537982,-27.99816,26.97,0.86,27.11,1.03,0.62,22.38,1.0,18.45,1.0,0.9,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,20.3227976339,0.0346211767183,False,20.317176206,0.0412507669089,False,20.5253497945,0.0485136820714,False,20.7350090738,0.0588474907728,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
8,300482,52.542172,-28.00188,12.58,0.69,11.28,0.94,0.15,14.26,0.93,11.69,0.96,0.64,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,21.1507983972,0.059551508846,False,21.2692272509,0.0904780170632,False,21.0147011862,0.070808882919,False,21.2304637221,0.0891622546251,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False
9,300691,52.536442,-27.99719,3.4,0.56,3.66,0.58,0.42,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,22.5713027074,0.178827139607,False,22.4912972865,0.172056557038,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False


### I.d -  PS1 3PSS  

This catalogue comes from `dmu0_SpARCS`. Alexandru Tudorica confirmed that the magnitudes are AB ones and are not aperture corrected.

In the catalogue, we keep:

- The internal identifier (this one is only in HeDaM data);
- The position;
- The ugrz magnitudes in the 8th aperture (11×0.186=2.046 arcsec).
- The "auto" magnitudes.

Note that there are y band columns because we combined all the SpARCS data in HeDaM, but there is no y data for the ELAIS-N1 sources.

The maps on the web page indicate they were observed in 2012 (or late 2011). Let's use 2012 as epoch.

There is a second notebook `sparcs_aperture_correction.ipynb` detailing how we choose some values for the aperture correction.

In [15]:
#Use r band radec because readme states that is only band with radec for every source
#Can not find stallarity - perhaps this is a flag id?
panstarrs = Table.read("../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_CDFS-SWIRE.fits")[
    'uniquePspsSTid', 'rra', 'rdec',   
    'gApMag', 'gApMagErr', 'gKronMag', 'gKronMagErr', 
    'rApMag', 'rApMagErr', 'rKronMag', 'rKronMagErr',
    'iApMag', 'iApMagErr', 'iKronMag', 'iKronMagErr',
    'zApMag', 'zApMagErr', 'zKronMag', 'zKronMagErr',
    'yApMag', 'yApMagErr', 'yKronMag', 'yKronMagErr']

panstarrs = Table(panstarrs.as_array(), names=[
    'panstarrs_intid', 'panstarrs_ra', 'panstarrs_dec',
    'm_app_panstarrs_g', 'merr_app_panstarrs_g', 'm_panstarrs_g', 'merr_panstarrs_g', 
    'm_app_panstarrs_r', 'merr_app_panstarrs_r', 'm_panstarrs_r', 'merr_panstarrs_r', 
    'm_app_panstarrs_i', 'merr_app_panstarrs_i', 'm_panstarrs_i', 'merr_panstarrs_i', 
    'm_app_panstarrs_z', 'merr_app_panstarrs_z', 'm_panstarrs_z', 'merr_panstarrs_z', 
    'm_app_panstarrs_y', 'merr_app_panstarrs_y', 'm_panstarrs_y', 'merr_panstarrs_y'
])
panstarrs_epoch = 2009 #table says e.g. 55962.33945
panstarrs_moc = MOC(filename="../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_CDFS-SWIRE_MOC.fits")

# Replace values with -999. with NaN
#panstarrs[panstarrs == -999.] = np.nan #fancy numpy doesn't work
for col in panstarrs.colnames:
    if col.startswith('m'): # | col.endswith('ra') | col.endswith('dec'):
        panstarrs[col][np.where(panstarrs[col] == -999.)] = np.nan

# Adding flux and band-flag columns
for col in panstarrs.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        flux, error = utils.mag_to_flux(np.array(panstarrs[col]), np.array(panstarrs[errcol]))

        # Fluxes are added in µJy
        panstarrs.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        panstarrs.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        panstarrs.add_column(Column(np.zeros(len(panstarrs), dtype=bool), name="flag{}".format(col[1:])))
        

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


In [16]:
panstarrs[:10].show_in_notebook()

idx,panstarrs_intid,panstarrs_ra,panstarrs_dec,m_app_panstarrs_g,merr_app_panstarrs_g,m_panstarrs_g,merr_panstarrs_g,m_app_panstarrs_r,merr_app_panstarrs_r,m_panstarrs_r,merr_panstarrs_r,m_app_panstarrs_i,merr_app_panstarrs_i,m_panstarrs_i,merr_panstarrs_i,m_app_panstarrs_z,merr_app_panstarrs_z,m_panstarrs_z,merr_panstarrs_z,m_app_panstarrs_y,merr_app_panstarrs_y,m_panstarrs_y,merr_panstarrs_y,f_app_panstarrs_g,ferr_app_panstarrs_g,flag_app_panstarrs_g,f_panstarrs_g,ferr_panstarrs_g,flag_panstarrs_g,f_app_panstarrs_r,ferr_app_panstarrs_r,flag_app_panstarrs_r,f_panstarrs_r,ferr_panstarrs_r,flag_panstarrs_r,f_app_panstarrs_i,ferr_app_panstarrs_i,flag_app_panstarrs_i,f_panstarrs_i,ferr_panstarrs_i,flag_panstarrs_i,f_app_panstarrs_z,ferr_app_panstarrs_z,flag_app_panstarrs_z,f_panstarrs_z,ferr_panstarrs_z,flag_panstarrs_z,f_app_panstarrs_y,ferr_app_panstarrs_y,flag_app_panstarrs_y,f_panstarrs_y,ferr_panstarrs_y,flag_panstarrs_y
0,1044575000001629,52.52906269,-28.00194851,23.5210990906,0.021276999265,nan,nan,nan,0.012261999771,21.4883995056,0.123677000403,nan,0.00909699965268,nan,nan,21.6175994873,0.00822700001299,nan,nan,20.8388996124,0.005795000121,nan,nan,1.41762173685,0.0277809071123,False,nan,nan,False,nan,nan,False,9.21807416052,1.05003752907,False,nan,nan,False,nan,nan,False,8.18389794665,0.0620122348489,False,nan,nan,False,16.7664128178,0.0894889236283,False,nan,nan,False
1,1044575000000385,52.5315548,-28.00073337,nan,0.0181089993566,nan,nan,nan,0.00812599994242,21.9500007629,0.163737997413,nan,0.0118119996041,nan,nan,23.0020008087,0.0155649995431,nan,nan,nan,0.0148600004613,nan,nan,nan,nan,False,nan,nan,False,nan,nan,False,6.0255916266,0.908709041701,False,nan,nan,False,nan,nan,False,2.2866498997,0.0327811714191,False,nan,nan,False,nan,nan,False,nan,nan,False
2,1044031000004158,52.53204583,-28.00106594,nan,0.0184179991484,nan,nan,nan,0.0889360010624,nan,nan,23.0331001282,0.0100750001147,22.9598007202,0.302852988243,21.5739002228,0.00722299981862,nan,nan,nan,0.0197410006076,nan,nan,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,2.2220813213,0.020619621478,False,2.37727658017,0.663112561807,False,8.52000620169,0.056680437642,False,nan,nan,False,nan,nan,False,nan,nan,False
3,1044575000001637,52.53203882,-28.00106933,nan,0.019756000489,nan,nan,nan,0.00677399989218,nan,nan,22.8180999756,0.00951300002635,22.7593002319,0.246711999178,21.5007991791,0.00779600022361,nan,nan,23.3540992737,0.0184550005943,nan,nan,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,2.70869439931,0.0237330299746,False,2.85943288329,0.649749359061,False,9.11339781509,0.0654376736199,False,nan,nan,False,1.6533328337,0.0281028285643,False,nan,nan,False
4,1044031000001432,52.53986429,-28.00321425,22.7817001343,0.0133130000904,nan,nan,22.9813995361,0.0137729998678,nan,nan,21.7229995728,0.00551000004634,22.0044002533,0.134818002582,23.3511009216,0.0163739994168,nan,nan,22.9706993103,0.0138830002397,nan,nan,2.80104409886,0.0343456358717,False,nan,nan,False,2.33045212498,0.029562721284,False,nan,nan,False,7.42677335964,0.0376901142068,False,5.73112523546,0.711645106245,False,1.65790496194,0.0250028826164,False,nan,nan,False,2.35353291191,0.0300939563746,False,nan,nan,False
5,1044575000000405,52.53986334,-28.00321892,22.6770000458,0.0144239999354,nan,nan,nan,0.0168180000037,nan,nan,21.7572002411,0.00583600020036,21.9636001587,0.136280998588,23.129699707,0.0165069997311,nan,nan,21.5256004333,0.00795099977404,nan,nan,3.08460800771,0.0409790016294,False,nan,nan,False,nan,nan,False,nan,nan,False,7.19647636771,0.0386821746754,False,5.9505887454,0.746914557011,False,2.03291919807,0.0309075045198,False,nan,nan,False,8.907581952,0.0652314823593,False,nan,nan,False
6,1044031000004216,52.54063684,-28.00058516,24.0830993652,0.0242420006543,nan,nan,22.4395008087,0.0107310004532,22.4022006989,0.234465003014,22.4631004333,0.00774799985811,22.809299469,0.268458008766,22.5202999115,0.0111689995974,22.6072006226,0.352306991816,22.2401008606,0.00991700030863,nan,nan,

### 1.e - Fireworks 

In [28]:
#Based on PanSTARRS
fireworks = Table.read("../../dmu0/dmu0_Fireworks/data/fireworks.fits")[
    'Seq', 'RAJ2000', 'DEJ2000',   
    'FU38', 'e_FU38', 
    'FB435', 'e_FB435', 
    'FB', 'e_FB',
    'FV', 'e_FV',
    'FV606', 'e_FV606',
    'FRc', 'e_FRc',
    'Fi775', 'e_Fi775',
    'FI', 'e_FI',
    'Fz850', 'e_Fz850',
    'FJ', 'e_FJ',
    'FH', 'e_FH',
    'FKs', 'e_FKs',
    'F3.6', 'e_F3.6',
    'F4.5', 'e_F4.5',
    'F5.8', 'e_F5.8',
    'F8.0', 'e_F8.0',
    'FKs.t', 'e_FKs.t',
    'F24.t', 'e_F24.t'
    ]

#Are these aperture magnitudes - in the readme only Ks band fluxes have an associated apperture
fireworks = Table(fireworks.as_array(), names=[
    'fireworks_intid', 'fireworks_ra', 'fireworks_dec',
    'm_fireworks_FU38',  'merr_fireworks_FU38', 
    'm_fireworks_FB435', 'merr_fireworks_FB435', 
    'm_fireworks_FB',    'merr_fireworks_FB',
    'm_fireworks_FV',    'merr_fireworks_FV',
    'm_fireworks_FV606', 'merr_fireworks_FV606',
    'm_fireworks_FRc',   'merr_fireworks_FRc',
    'm_fireworks_Fi775', 'merr_fireworks_Fi775',
    'm_fireworks_FI',    'merr_fireworks_FI',
    'm_fireworks_Fz850', 'merr_fireworks_Fz850',
    'm_fireworks_FJ',    'merr_fireworks_FJ',
    'm_fireworks_FH',    'merr_fireworks_FH',
    'm_fireworks_FKs',   'merr_fireworks_FKs',
    'm_fireworks_F3.6',  'merr_fireworks_F3.6',
    'm_fireworks_F4.5',  'merr_fireworks_F4.5',
    'm_fireworks_F5.8',  'merr_fireworks_F5.8',
    'm_fireworks_F8.0',  'merr_fireworks_F8.0',
    'm_fireworks_FKs.t', 'merr_fireworks_FKs.t',
    'm_fireworks_F24.t', 'merr_fireworks_F24.t'
])
fireworks_epoch = 2008 #Paper is from 2008 - Input surveys will have individual epochs
fireworks_moc = MOC(filename="../../dmu0/dmu0_Fireworks/data/fireworks_MOC.fits")


# Add flux and band-flag columns
for col in fireworks.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        flux, error = utils.mag_to_flux(np.array(fireworks[col]), np.array(fireworks[errcol]))

        # Fluxes are added in µJy
        fireworks.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        fireworks.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        fireworks.add_column(Column(np.zeros(len(fireworks), dtype=bool), name="flag{}".format(col[1:])))

In [29]:
fireworks[:10].show_in_notebook()

idx,fireworks_intid,fireworks_ra,fireworks_dec,m_fireworks_FU38,merr_fireworks_FU38,m_fireworks_FB435,merr_fireworks_FB435,m_fireworks_FB,merr_fireworks_FB,m_fireworks_FV,merr_fireworks_FV,m_fireworks_FV606,merr_fireworks_FV606,m_fireworks_FRc,merr_fireworks_FRc,m_fireworks_Fi775,merr_fireworks_Fi775,m_fireworks_FI,merr_fireworks_FI,m_fireworks_Fz850,merr_fireworks_Fz850,m_fireworks_FJ,merr_fireworks_FJ,m_fireworks_FH,merr_fireworks_FH,m_fireworks_FKs,merr_fireworks_FKs,m_fireworks_F3.6,merr_fireworks_F3.6,m_fireworks_F4.5,merr_fireworks_F4.5,m_fireworks_F5.8,merr_fireworks_F5.8,m_fireworks_F8.0,merr_fireworks_F8.0,m_fireworks_FKs.t,merr_fireworks_FKs.t,m_fireworks_F24.t,merr_fireworks_F24.t,f_fireworks_FU38,ferr_fireworks_FU38,flag_fireworks_FU38,f_fireworks_FB435,ferr_fireworks_FB435,flag_fireworks_FB435,f_fireworks_FB,ferr_fireworks_FB,flag_fireworks_FB,f_fireworks_FV,ferr_fireworks_FV,flag_fireworks_FV,f_fireworks_FV606,ferr_fireworks_FV606,flag_fireworks_FV606,f_fireworks_FRc,ferr_fireworks_FRc,flag_fireworks_FRc,f_fireworks_Fi775,ferr_fireworks_Fi775,flag_fireworks_Fi775,f_fireworks_FI,ferr_fireworks_FI,flag_fireworks_FI,f_fireworks_Fz850,ferr_fireworks_Fz850,flag_fireworks_Fz850,f_fireworks_FJ,ferr_fireworks_FJ,flag_fireworks_FJ,f_fireworks_FH,ferr_fireworks_FH,flag_fireworks_FH,f_fireworks_FKs,ferr_fireworks_FKs,flag_fireworks_FKs,f_fireworks_F3.6,ferr_fireworks_F3.6,flag_fireworks_F3.6,f_fireworks_F4.5,ferr_fireworks_F4.5,flag_fireworks_F4.5,f_fireworks_F5.8,ferr_fireworks_F5.8,flag_fireworks_F5.8,f_fireworks_F8.0,ferr_fireworks_F8.0,flag_fireworks_F8.0,f_fireworks_FKs.t,ferr_fireworks_FKs.t,flag_fireworks_FKs.t,f_fireworks_F24.t,ferr_fireworks_F24.t,flag_fireworks_F24.t
0,1,53.2033631,-27.9372714,0.379,0.03749,0.07,nan,0.536,0.01313,0.721,0.01757,1.179,0.04691,1.445,0.01582,2.636,0.06975,2.337,0.09936,3.067,0.09619,4.073,0.202,0.163,14.31,6.378,0.34084,6.152,0.07331,3.937,0.10984,3.846,0.77398,2.242,0.66438,7.421,0.3966,-20.498,7.01084,2560943517.16,88428268.3376,False,3404081897.01,nan,False,2216154325.96,26800346.3202,False,1868959971.65,30244571.8933,False,1225744632.05,52959163.043,False,959400631.516,13979194.8763,False,320331760.085,20578792.6828,False,421890746.568,38608885.2847,False,215377335.566,19081196.535,False,85270733.7344,15864524.1255,False,3124640085.81,41182747182.9,False,10204694.3194,3203511.12599,False,12566085.2008,848474.705026,False,96649586.4314,9777688.65688,False,105099342.719,74921319.6731,False,460468578.379,281768363.957,False,3904810.83749,1426357.49951,False,5.74380913496e+17,3.70890522572e+18,False
1,2,53.2103094,-27.937425,0.295,0.04015,0.148,nan,1.008,0.01386,1.655,0.01861,2.027,0.04063,2.418,0.01656,3.004,0.05869,3.398,0.10536,3.512,0.08694,3.946,0.276,-0.171,14.31,5.323,0.53908,4.408,0.08874,3.763,0.12677,-0.363,0.0,1.979,0.74863,5.976,0.60521,-0.363,0.0,2766941645.41,102320164.49,False,3168108159.33,nan,False,1434828518.23,18316349.0133,False,790678628.0,13552582.2963,False,561306407.675,21004991.1239,False,391561514.84,5972222.95556,False,228244331.217,12337858.6257,False,158781536.421,15408183.5016,False,142955214.004,11447095.7596,False,95851739.7686,24366029.299,False,4250108331.76,56016415369.6,False,26964973.6354,13388406.8006,False,62632536.4758,5119117.57433,False,113448824.421,13246226.3014,False,5072242398.93,0.0,False,586678266.402,404522708.787,False,14777467.0617,8237241.05471,False,5072242398.93,0.0,False
2,3,53.2042533,-27.9371053,0.239,0.04178,0.109,nan,2.189,0.01459,5.447,0.01962,8.797,0.04467,13.027,0.01763,30.494,0.06376,42.097,0.11064,47.728,0.09499,66.075,0.249,0.163,14.31,58.354,0.4049,35.166,0.07526,24.012,0.11089,14.707,0.76757,8.857,0.66592,64.601,0.44825,-11.14,6.91829,2913399222.57,112109938.847,False,3283976192.5,nan,False,483503920.592,6497270.85688,False,24054703.0499,434685.029172,False,1099512.06688,45236.7746532,False,22346.0105773,362.850722626,False,0.00230356250012,0.00013527700777,False,5.2625956208e-08,5.36275364986e-09,False,2.9430659897e-1

### 1.f - COMBO-17

In [35]:
#Based on PanSTARRS
combo = Table.read("../../dmu0/dmu0_COMBO-17/data/table3.fits")[
    'Seq', 'ra', 'dec',  'stellarity',
    'Rmag',    'e_Rmag',  
    'UjMag',   'e_UjMag', 
    'BjMag',   'e_BjMag',
    'VjMag',   'e_VjMag',
    'usMag',   'e_usMag',
    'gsMag',   'e_gsMag',
    'rsMag',   'e_rsMag',
    'UbMag',   'e_UbMag',
    'BbMag',   'e_BbMag',
    'VbMag',   'e_VbMag',
    'S280Mag', 'e_S280Mag',
    'S145Mag', 'e_S145Mag'
]

combo = Table(combo.as_array(), names=[
    'combo_intid', 'combo_ra', 'combo_dec', 'combo_stellarity',
    'm_combo_R',    'merr_combo_R',  
    'm_combo_Uj',   'merr_combo_Uj', 
    'm_combo_Bj',   'merr_combo_Bj',
    'm_combo_Vj',   'merr_combo_Vj',
    'm_combo_us',   'merr_combo_us',
    'm_combo_gs',   'merr_combo_gs',
    'm_combo_rs',   'merr_combo_rs',
    'm_combo_Ub',   'merr_combo_Ub',
    'm_combo_Bb',   'merr_combo_Bb',
    'm_combo_Vb',   'merr_combo_Vb',
    'm_combo_S280', 'merr_combo_S280',
    'm_combo_S145', 'merr_combo_S145'
])
combo_epoch = 2009 #table says e.g. 55962.33945
combo_moc = MOC(filename="../../dmu0/dmu0_COMBO-17/data/table3_MOC.fits")

#Replace 0.0 with NaN values
for col in panstarrs.colnames:
    if col.startswith('m'): # | col.endswith('ra') | col.endswith('dec'):
        panstarrs[col][np.where(panstarrs[col] == 0.0)] = np.nan

# Add flux and band-flag columns
for col in combo.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        flux, error = utils.mag_to_flux(np.array(combo[col]), np.array(combo[errcol]))

        # Fluxes are added in µJy
        combo.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        combo.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        combo.add_column(Column(np.zeros(len(combo), dtype=bool), name="flag{}".format(col[1:])))

In [36]:
combo[:10].show_in_notebook()

idx,combo_intid,combo_ra,combo_dec,combo_stellarity,m_combo_R,merr_combo_R,m_combo_Uj,merr_combo_Uj,m_combo_Bj,merr_combo_Bj,m_combo_Vj,merr_combo_Vj,m_combo_us,merr_combo_us,m_combo_gs,merr_combo_gs,m_combo_rs,merr_combo_rs,m_combo_Ub,merr_combo_Ub,m_combo_Bb,merr_combo_Bb,m_combo_Vb,merr_combo_Vb,m_combo_S280,merr_combo_S280,m_combo_S145,merr_combo_S145,f_combo_R,ferr_combo_R,flag_combo_R,f_combo_Uj,ferr_combo_Uj,flag_combo_Uj,f_combo_Bj,ferr_combo_Bj,flag_combo_Bj,f_combo_Vj,ferr_combo_Vj,flag_combo_Vj,f_combo_us,ferr_combo_us,flag_combo_us,f_combo_gs,ferr_combo_gs,flag_combo_gs,f_combo_rs,ferr_combo_rs,flag_combo_rs,f_combo_Ub,ferr_combo_Ub,flag_combo_Ub,f_combo_Bb,ferr_combo_Bb,flag_combo_Bb,f_combo_Vb,ferr_combo_Vb,flag_combo_Vb,f_combo_S280,ferr_combo_S280,flag_combo_S280,f_combo_S145,ferr_combo_S145,flag_combo_S145
0,7864,52.9043509407,-28.004407161,0.704,25.9685,0.2545,-16.8335858104,0.289280533791,-16.1213024315,0.939844787121,-16.41656581,0.939844787121,-16.9804966689,0.289280533791,-16.2014578601,0.939844787121,-16.6361515477,0.939844787121,0.0,0.0,0.0,0.0,0.0,0.0,-16.8034956128,0.609699368477,nan,nan,0.148806,0.0348805,False,1.96532475335e+16,5.23635659592e+15,False,1.01981400693e+16,8.82780618392e+15,False,1.33852449747e+16,1.15866567392e+16,False,2.25008366885e+16,5.99506032816e+15,False,1.09795146886e+16,9.50418674442e+15,False,1.63855347398e+16,1.41837946841e+16,False,3630780547.7,0.0,False,3630780547.7,0.0,False,3630780547.7,0.0,False,1.91160538442e+16,1.07346940311e+16,False,nan,nan,False
1,7867,52.9248830635,-28.0045354786,0.659,25.7862,0.2353,-17.6367689613,0.273388147354,-17.4349329199,0.523457169533,-17.798817823,0.523457169533,-17.7750868334,0.273388147354,-17.5348377222,0.523457169533,-18.0605224591,0.523457169533,0.0,0.0,0.0,0.0,0.0,0.0,-18.2183962955,0.354991674423,nan,nan,0.175997,0.038142,False,4.11820142481e+16,1.03696225015e+16,False,3.41958314623e+16,1.64865582161e+16,False,4.7810923141e+16,2.30506916786e+16,False,4.67772550611e+16,1.17785029581e+16,False,3.74916961709e+16,1.80755666732e+16,False,6.08427707855e+16,2.93336304365e+16,False,3630780547.7,0.0,False,3630780547.7,0.0,False,3630780547.7,0.0,False,7.03652958866e+16,2.3006595984e+16,False,nan,nan,False
2,7868,52.8874689235,-28.0044207329,0.15,25.622,0.2452,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,nan,nan,nan,nan,0.20473,0.0462358,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,nan,nan,False,3630780547.7,0.0,False,3630780547.7,0.0,False,3630780547.7,0.0,False,nan,nan,False,nan,nan,False
3,7875,52.8439537238,-28.00426117,0.673,25.548,0.2829,-17.5415660518,0.31585970521,-17.3753730331,0.570135295391,-17.7927575649,0.570135295391,-17.6741450974,0.31585970521,-17.4914362224,0.570135295391,-18.0691218723,0.570135295391,0.0,0.0,0.0,0.0,0.0,0.0,-17.8953102303,0.443313926458,nan,nan,0.219185,0.0571109,False,3.77247543695e+16,1.09747927186e+16,False,3.23704855089e+16,1.69981955439e+16,False,4.75448001198e+16,2.49664407817e+16,False,4.26243764695e+16,1.24001787243e+16,False,3.60225529976e+16,1.89159473581e+16,False,6.1326580402e+16,3.220344673e+16,False,3630780547.7,0.0,False,3630780547.7,0.0,False,3630780547.7,0.0,False,5.22545475972e+16,2.13359088231e+16,False,nan,nan,False
4,7885,53.0014692079,-28.0045796817,0.811,25.4399,0.1976,-13.6272840924,0.273518174887,-13.2421196937,0.288551092148,-13.1350464832,0.240335285664,-13.8384368202,0.273518174887,-13.0698528306,0.240335285664,-13.3800255777,0.240335285664,0.0,0.0,0.0,0.0,0.0,0.0,-14.6258954687,0.333708137274,nan,nan,0.242129,0.0440666,False,1.02544798722e+15,2.58330394322e+14,False,7.1919701713e+14,1.91137666638e+14,False,6.5165629257e+14,1.44248667902e+14,False,1.24558889621e+15,3.13788192803e+14,False,6.13678816593e+14,1.35842088571e+14,False,8.1660160856e+14,1.80760464657e+14,False,3630780547.7,0.0,False,3630780547.7,0.0,False,3630780547.7,0.0,False,2.5725166096e+15,7.90679837454e+14,False,nan,nan,False
5,7887,52.8518140423,-28.0042462

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [38]:
nb_video_orig = len(video)
video = masterlist.remove_duplicates(
    video, 'video_ra', 'video_dec', 
    sort_col=['merr_app_video_z', 'merr_app_video_y', 'merr_app_video_j', 'merr_app_video_h', 'merr_app_video_k'],
    flag_name='video_flag_cleaned')
nb_video = len(video)
print("VIDEO initial catalogue as {} sources.".format(nb_video_orig))
print("VIDEO cleaned catalogue as {} sources ({} removed).".format(nb_video, nb_video_orig - nb_video))
print("VIDEO has {} sources flagged as having been cleaned".format(np.sum(video['video_flag_cleaned'])))

VIDEO initial catalogue as 190847 sources.
VIDEO cleaned catalogue as 190396 sources (451 removed).
VIDEO has 449 sources flagged as having been cleaned


In [39]:
nb_servs_orig = len(servs)
servs = masterlist.remove_duplicates(
    servs, 'servs_ra', 'servs_dec', 
    sort_col=['ferr_app_servs_irac1', 'ferr_app_servs_irac2'],
    flag_name='servs_flag_cleaned')
nb_servs = len(servs)
print("SERVS initial catalogue as {} sources.".format(nb_servs_orig))
print("SERVS cleaned catalogue as {} sources ({} removed).".format(nb_servs, nb_servs_orig - nb_servs))
print("SERVS has {} sources flagged as having been cleaned".format(np.sum(servs['servs_flag_cleaned'])))

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


SERVS initial catalogue as 152497 sources.
SERVS cleaned catalogue as 152497 sources (0 removed).
SERVS has 0 sources flagged as having been cleaned


In [40]:
nb_swire_orig = len(swire)
swire = masterlist.remove_duplicates(
    swire, 'swire_ra', 'swire_dec', 
    sort_col=['ferr_app_swire_irac1', 'ferr_app_swire_irac2', 
              'ferr_app_irac3', 'ferr_app_irac4'],
    flag_name='swire_flag_cleaned')
nb_swire = len(swire)
print("SWIRE initial catalogue as {} sources.".format(nb_swire_orig))
print("SWIRE cleaned catalogue as {} sources ({} removed).".format(nb_swire, nb_swire_orig - nb_swire))
print("SWIRE has {} sources flagged as having been cleaned".format(np.sum(swire['swire_flag_cleaned'])))

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


SWIRE initial catalogue as 56221 sources.
SWIRE cleaned catalogue as 56221 sources (0 removed).
SWIRE has 0 sources flagged as having been cleaned


In [42]:
nb_panstarrs_orig = len(panstarrs)
panstarrs = masterlist.remove_duplicates(
    panstarrs, 'panstarrs_ra', 'panstarrs_dec', 
    sort_col=['merr_app_panstarrs_g', 
              'merr_app_panstarrs_r',
              'merr_app_panstarrs_i', 
              'merr_app_panstarrs_z', 
              'merr_app_panstarrs_y'],
    flag_name='panstarrs_flag_cleaned')
nb_panstarrs = len(panstarrs)
print("PanSTARRS initial catalogue as {} sources.".format(nb_panstarrs_orig))
print("PanSTARRS cleaned catalogue as {} sources ({} removed).".format(nb_panstarrs, nb_panstarrs_orig - nb_panstarrs))
print("PanSTARRS has {} sources flagged as having been cleaned".format(np.sum(panstarrs['panstarrs_flag_cleaned'])))

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


PanSTARRS initial catalogue as 43975 sources.
PanSTARRS cleaned catalogue as 37491 sources (6484 removed).
PanSTARRS has 5897 sources flagged as having been cleaned


In [45]:
nb_fireworks_orig = len(fireworks)
fireworks = masterlist.remove_duplicates(
    fireworks, 'fireworks_ra', 'fireworks_dec', 
    sort_col=['merr_fireworks_FU38', 
     'merr_fireworks_FB435', 
     'merr_fireworks_FB',
     'merr_fireworks_FV',
     'merr_fireworks_FV606',
     'merr_fireworks_FRc',
     'merr_fireworks_Fi775',
     'merr_fireworks_FI',
     'merr_fireworks_Fz850',
     'merr_fireworks_FJ',
     'merr_fireworks_FH',
     'merr_fireworks_FKs',
     'merr_fireworks_F3.6',
     'merr_fireworks_F4.5',
     'merr_fireworks_F5.8',
     'merr_fireworks_F8.0',
     'merr_fireworks_FKs.t',
     'merr_fireworks_F24.t'],
    flag_name='fireworks_flag_cleaned')
nb_fireworks = len(fireworks)
print("Fireworks initial catalogue as {} sources.".format(nb_fireworks_orig))
print("Fireworks cleaned catalogue as {} sources ({} removed).".format(nb_fireworks, nb_fireworks_orig - nb_fireworks))
print("Fireworks has {} sources flagged as having been cleaned".format(np.sum(fireworks['fireworks_flag_cleaned'])))

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


ValueError: Duplicate column names

In [46]:
nb_combo_orig = len(combo)
combo = masterlist.remove_duplicates(
    combo, 'combo_ra', 'combo_dec', 
    sort_col=['merr_combo_R',  
     'merr_combo_Uj', 
     'merr_combo_Bj',
     'merr_combo_Vj',
     'merr_combo_us',
     'merr_combo_gs',
     'merr_combo_rs',
     'merr_combo_Ub',
     'merr_combo_Bb',
     'merr_combo_Vb',
     'merr_combo_S280',
     'merr_combo_S145'],
    flag_name='combo_flag_cleaned')
nb_combo = len(combo)
print("COMBO initial catalogue as {} sources.".format(nb_combo_orig))
print("COMBO cleaned catalogue as {} sources ({} removed).".format(nb_combo, nb_combo_orig - nb_combo))
print("COMBO has {} sources flagged as having been cleaned".format(np.sum(combo['combo_flag_cleaned'])))

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


COMBO initial catalogue as 55551 sources.
COMBO cleaned catalogue as 55551 sources (0 removed).
COMBO has 0 sources flagged as having been cleaned


## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [47]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_ELAIS-N1.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [48]:
masterlist.nb_astcor_diag_plot(
    wfc['wfc_ra'], wfc['wfc_dec'], gaia_coords.ra, gaia_coords.dec)

NameError: name 'wfc' is not defined

In [ ]:
wfc_delta_ra, wfc_delta_dec = utils.astrometric_correction(
    SkyCoord(wfc['wfc_ra'], wfc['wfc_dec']),
    gaia_coords
)
wfc['wfc_ra'] += wfc_delta_ra.to(u.deg)
wfc['wfc_dec'] += wfc_delta_dec.to(u.deg)

print("WFC delta RA / delta Dec: {} / {}".format(wfc_delta_ra, wfc_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    dxs['dxs_ra'], dxs['dxs_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
dxs_delta_ra, dxs_delta_dec = utils.astrometric_correction(
    SkyCoord(dxs['dxs_ra'], dxs['dxs_dec']),
    gaia_coords
)
dxs['dxs_ra'] += dxs_delta_ra.to(u.deg)
dxs['dxs_dec'] += dxs_delta_dec.to(u.deg)

print("DXS delta RA / delta Dec: {} / {}".format(dxs_delta_ra, dxs_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    servs['servs_ra'], servs['servs_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
servs_delta_ra, servs_delta_dec = utils.astrometric_correction(
    SkyCoord(servs['servs_ra'], servs['servs_dec']),
    gaia_coords
)
servs['servs_ra'] += servs_delta_ra.to(u.deg)
servs['servs_dec'] += servs_delta_dec.to(u.deg)

print("SERVS delta RA / delta Dec: {} / {}".format(servs_delta_ra, servs_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    swire['swire_ra'], swire['swire_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
swire_delta_ra, swire_delta_dec = utils.astrometric_correction(
    SkyCoord(swire['swire_ra'], swire['swire_dec']),
    gaia_coords
)
swire['swire_ra'] += swire_delta_ra.to(u.deg)
swire['swire_dec'] += swire_delta_dec.to(u.deg)

print("SWIRE delta RA / delta Dec: {} / {}".format(swire_delta_ra, swire_delta_dec))

In [ ]:
masterlist.nb_astcor_diag_plot(
    sparcs['sparcs_ra'], sparcs['sparcs_dec'], gaia_coords.ra, gaia_coords.dec)

In [ ]:
sparcs_delta_ra, sparcs_delta_dec = utils.astrometric_correction(
    SkyCoord(sparcs['sparcs_ra'], sparcs['sparcs_dec']),
    gaia_coords
)
sparcs['sparcs_ra'] += sparcs_delta_ra.to(u.deg)
sparcs['sparcs_dec'] += sparcs_delta_dec.to(u.deg)

print("SpARCS delta RA / delta Dec: {} / {}".format(sparcs_delta_ra, sparcs_delta_dec))

## IV - Flagging Gaia objects

In [ ]:
wfc.add_column(
    flagging.gaia_flag_column(
        SkyCoord(wfc['wfc_ra'], wfc['wfc_dec']),
        wfc_epoch,
        gaia
    )
)
wfc['flag_gaia'].name = 'wfc_flag_gaia'
print("{} sources flagged.".format(np.sum(wfc['wfc_flag_gaia'] > 0)))

In [ ]:
dxs.add_column(
    flagging.gaia_flag_column(
        SkyCoord(dxs['dxs_ra'], dxs['dxs_dec']),
        dxs_epoch,
        gaia
    )
)
dxs['flag_gaia'].name = 'dxs_flag_gaia'
print("{} sources flagged.".format(np.sum(dxs['dxs_flag_gaia'] > 0)))

In [ ]:
servs.add_column(
    flagging.gaia_flag_column(
        SkyCoord(servs['servs_ra'], servs['servs_dec']),
        servs_epoch,
        gaia
    )
)
servs['flag_gaia'].name = 'servs_flag_gaia'
print("{} sources flagged.".format(np.sum(servs['servs_flag_gaia'] > 0)))

In [ ]:
swire.add_column(
    flagging.gaia_flag_column(
        SkyCoord(swire['swire_ra'], swire['swire_dec']),
        swire_epoch,
        gaia
    )
)
swire['flag_gaia'].name = 'swire_flag_gaia'
print("{} sources flagged.".format(np.sum(swire['swire_flag_gaia'] > 0)))

In [ ]:
sparcs.add_column(
    flagging.gaia_flag_column(
        SkyCoord(sparcs['sparcs_ra'], sparcs['sparcs_dec']),
        sparcs_epoch,
        gaia
    )
)
sparcs['flag_gaia'].name = 'sparcs_flag_gaia'
print("{} sources flagged.".format(np.sum(sparcs['sparcs_flag_gaia'] > 0)))

## IV - Flagging objects near bright stars

In [ ]:
#This section may be superseeded by merely using multiband detection as evidence of good data
#starmask.flag_artefacts

## V- Merging the catalogues

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(wfc['wfc_ra'], wfc['wfc_dec']),
    SkyCoord(dxs['dxs_ra'], dxs['dxs_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
wfc['wfc_ra'].name = 'ra'
wfc['wfc_dec'].name = 'dec'
masterlist_catalogue = masterlist.merge_catalogues(
    wfc, dxs, "dxs_ra", "dxs_dec")

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    SkyCoord(servs['servs_ra'], servs['servs_dec'])
)

In [ ]:
# Given the graph above, we use 1.2 arc-second radius
masterlist_catalogue = masterlist.merge_catalogues(
    masterlist_catalogue, servs, "servs_ra", "servs_dec")

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    SkyCoord(swire['swire_ra'], swire['swire_dec'])
)

In [ ]:
masterlist_catalogue = masterlist.merge_catalogues(
    masterlist_catalogue, swire, "swire_ra", "swire_dec")

In [ ]:
masterlist.nb_merge_dist_plot(
    SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    SkyCoord(sparcs['sparcs_ra'], sparcs['sparcs_dec'])
)

In [ ]:
# Given the graph above, we use 1 arc-second radius
masterlist_catalogue = masterlist.merge_catalogues(
    masterlist_catalogue, sparcs, "sparcs_ra", "sparcs_dec", radius=1. * u.arcsec)

In [ ]:
# When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes
# only from a catalogue and has no counterparts in the other, the columns from the latest
# are masked for that row). We indicate to use NaN for masked values for floats columns,
# False for flag columns and -1 for ID columns.
for col in masterlist_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        masterlist_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        masterlist_catalogue[col].fill_value = False
    elif "id" in col:
        masterlist_catalogue[col].fill_value = -1
        
masterlist_catalogue = masterlist_catalogue.filled()

In [ ]:
masterlist_catalogue[:10].show_in_notebook()

## VI - Merging flags and stellarity

In [ ]:
masterlist_catalogue.add_column(Column(
    data=(masterlist_catalogue['wfc_flag_cleaned'] | 
          masterlist_catalogue['dxs_flag_cleaned'] |
          masterlist_catalogue['servs_flag_cleaned'] | 
          masterlist_catalogue['swire_flag_cleaned'] |
          masterlist_catalogue['sparcs_flag_cleaned']),
    name="flag_cleaned"
))
masterlist_catalogue.remove_columns(['wfc_flag_cleaned', 'dxs_flag_cleaned',
                                     'servs_flag_cleaned', 'swire_flag_cleaned',
                                     'sparcs_flag_cleaned'])

In [ ]:
masterlist_catalogue.add_column(Column(
    data=(masterlist_catalogue['wfc_flag_gaia'] | 
          masterlist_catalogue['dxs_flag_gaia'] |
          masterlist_catalogue['servs_flag_gaia'] | 
          masterlist_catalogue['swire_flag_gaia'] |
          masterlist_catalogue['sparcs_flag_gaia']),
    name="flag_gaia"
))
masterlist_catalogue.remove_columns(['wfc_flag_gaia', 'dxs_flag_gaia',
                                     'servs_flag_gaia', 'swire_flag_gaia',
                                     'sparcs_flag_gaia'])

In [ ]:
masterlist_catalogue.add_column(Column(
    data=np.nanmax([masterlist_catalogue['wfc_stellarity'],
                     masterlist_catalogue['dxs_stellarity'],
                     masterlist_catalogue['servs_stellarity_irac1'],
                     masterlist_catalogue['servs_stellarity_irac2'],
                     masterlist_catalogue['swire_stellarity_irac1'],
                     masterlist_catalogue['swire_stellarity_irac2'],
                     masterlist_catalogue['swire_stellarity_irac3'],
                     masterlist_catalogue['swire_stellarity_irac4'],
                     masterlist_catalogue['sparcs_stellarity']],
                   axis=0),
    name='stellarity'
))
masterlist_catalogue.remove_columns(
    ['wfc_stellarity',
     'dxs_stellarity',
     'servs_stellarity_irac1',
     'servs_stellarity_irac2',
     'swire_stellarity_irac1',
     'swire_stellarity_irac2',
     'swire_stellarity_irac3',
     'swire_stellarity_irac4',
     'sparcs_stellarity']
)

## VII - E(B-V)

In [ ]:
masterlist_catalogue.add_column(
    utils.ebv(masterlist_catalogue['ra'], masterlist_catalogue['dec'])
)

## VIII - Adding HELP unique identifiers and field columns

In [ ]:
masterlist_catalogue.add_column(Column(
    utils.gen_help_id(masterlist_catalogue['ra'], masterlist_catalogue['dec']),
    name="help_id"
))
masterlist_catalogue.add_column(Column(
    np.full(len(masterlist_catalogue), "ELAIS-N1", dtype='<U18'),
    name="field"
))

In [ ]:
# Check that the HELP Ids are unique
if len(masterlist_catalogue) != len(np.unique(masterlist_catalogue['help_id'])):
    print("The HELP IDs are not unique!!!")
else:
    print("OK!")

## IX - Choosing between multiple values for the same filter

Both SERVS and SWIRE provide IRAC1 and IRAC2 fluxes. SERVS is deeper but tends to under-estimate flux of bright sources (Mattia said over 2000 µJy) as illustrated by this comparison of SWIRE, SERVS, and Spitzer-EIP fluxes.

In [ ]:
seip = Table.read("../../dmu0/dmu0_SEIP/data/SEIP_ELAIS-N1.fits")
seip_coords = SkyCoord(seip['ra'], seip['dec'])
idx, d2d, _ = seip_coords.match_to_catalog_sky(SkyCoord(masterlist_catalogue['ra'], masterlist_catalogue['dec']))
mask = d2d <= 2 * u.arcsec

In [ ]:
fig, ax = plt.subplots()
ax.scatter(seip['i1_f_ap1'][mask], masterlist_catalogue[idx[mask]]['f_app_servs_irac1'], label="SERVS", s=2.)
ax.scatter(seip['i1_f_ap1'][mask], masterlist_catalogue[idx[mask]]['f_app_swire_irac1'], label="SWIRE", s=2.)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("SEIP flux [μJy]")
ax.set_ylabel("SERVS/SWIRE flux [μJy]")
ax.set_title("IRAC 1")
ax.legend()
ax.axvline(2000, color="black", linestyle="--", linewidth=1.)
ax.plot(seip['i1_f_ap1'][mask], seip['i1_f_ap1'][mask], linewidth=.1, color="black", alpha=.5)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(seip['i2_f_ap1'][mask], masterlist_catalogue[idx[mask]]['f_app_servs_irac2'], label="SERVS", s=2.)
ax.scatter(seip['i2_f_ap1'][mask], masterlist_catalogue[idx[mask]]['f_app_swire_irac2'], label="SWIRE", s=2.)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("SEIP flux [μJy]")
ax.set_ylabel("SERVS/SWIRE flux [μJy]")
ax.set_title("IRAC 2")
ax.legend()
ax.axvline(2000, color="black", linestyle="--", linewidth=1.)

ax.plot(seip['i1_f_ap2'][mask], seip['i1_f_ap2'][mask], linewidth=.1, color="black", alpha=.5)

When both SWIRE and SERVS fluxes are provided, we use the SERVS flux below 2000 μJy and the SWIRE flux over.

We create a table indicating for each source the origin on the IRAC1 and IRAC2 fluxes that will be saved separately.

In [ ]:
irac_origin = Table()
irac_origin.add_column(masterlist_catalogue['help_id'])

In [ ]:
# IRAC1 aperture flux and magnitudes
has_servs = ~np.isnan(masterlist_catalogue['f_app_servs_irac1'])
has_swire = ~np.isnan(masterlist_catalogue['f_app_swire_irac1'])
has_both = has_servs & has_swire

print("{} sources with SERVS flux".format(np.sum(has_servs)))
print("{} sources with SWIRE flux".format(np.sum(has_swire)))
print("{} sources with SERVS and SWIRE flux".format(np.sum(has_both)))

has_servs_above_limit = has_servs.copy()
has_servs_above_limit[has_servs] = masterlist_catalogue['f_app_servs_irac1'][has_servs] > 2000

use_swire = (has_swire & ~has_servs) | (has_both & has_servs_above_limit)
use_servs = (has_servs & ~(has_both & has_servs_above_limit))

print("{} sources for which we use SERVS".format(np.sum(use_servs)))
print("{} sources for which we use SWIRE".format(np.sum(use_swire)))

f_app_irac = np.full(len(masterlist_catalogue), np.nan)
f_app_irac[use_servs] = masterlist_catalogue['f_app_servs_irac1'][use_servs]
f_app_irac[use_swire] = masterlist_catalogue['f_app_swire_irac1'][use_swire]

ferr_app_irac = np.full(len(masterlist_catalogue), np.nan)
ferr_app_irac[use_servs] = masterlist_catalogue['ferr_app_servs_irac1'][use_servs]
ferr_app_irac[use_swire] = masterlist_catalogue['ferr_app_swire_irac1'][use_swire]

m_app_irac = np.full(len(masterlist_catalogue), np.nan)
m_app_irac[use_servs] = masterlist_catalogue['m_app_servs_irac1'][use_servs]
m_app_irac[use_swire] = masterlist_catalogue['m_app_swire_irac1'][use_swire]

merr_app_irac = np.full(len(masterlist_catalogue), np.nan)
merr_app_irac[use_servs] = masterlist_catalogue['merr_app_servs_irac1'][use_servs]
merr_app_irac[use_swire] = masterlist_catalogue['merr_app_swire_irac1'][use_swire]

masterlist_catalogue.add_column(Column(data=f_app_irac, name="f_app_irac1"))
masterlist_catalogue.add_column(Column(data=ferr_app_irac, name="ferr_app_irac1"))
masterlist_catalogue.add_column(Column(data=m_app_irac, name="m_app_irac1"))
masterlist_catalogue.add_column(Column(data=merr_app_irac, name="merr_app_irac1"))

masterlist_catalogue.remove_columns(['f_app_servs_irac1', 'f_app_swire_irac1', 'ferr_app_servs_irac1',
                                     'ferr_app_swire_irac1', 'm_app_servs_irac1', 'm_app_swire_irac1',
                                     'merr_app_servs_irac1', 'merr_app_swire_irac1'])

origin = np.full(len(masterlist_catalogue), '     ', dtype='<U5')
origin[use_servs] = "SERVS"
origin[use_swire] = "SWIRE"
irac_origin.add_column(Column(data=origin, name="IRAC1_app"))

In [ ]:
# IRAC1 total flux and magnitudes
has_servs = ~np.isnan(masterlist_catalogue['f_servs_irac1'])
has_swire = ~np.isnan(masterlist_catalogue['f_swire_irac1'])
has_both = has_servs & has_swire

print("{} sources with SERVS flux".format(np.sum(has_servs)))
print("{} sources with SWIRE flux".format(np.sum(has_swire)))
print("{} sources with SERVS and SWIRE flux".format(np.sum(has_both)))

has_servs_above_limit = has_servs.copy()
has_servs_above_limit[has_servs] = masterlist_catalogue['f_servs_irac1'][has_servs] > 2000

use_swire = (has_swire & ~has_servs) | (has_both & has_servs_above_limit)
use_servs = (has_servs & ~(has_both & has_servs_above_limit))

print("{} sources for which we use SERVS".format(np.sum(use_servs)))
print("{} sources for which we use SWIRE".format(np.sum(use_swire)))

f_app_irac = np.full(len(masterlist_catalogue), np.nan)
f_app_irac[use_servs] = masterlist_catalogue['f_servs_irac1'][use_servs]
f_app_irac[use_swire] = masterlist_catalogue['f_swire_irac1'][use_swire]

ferr_app_irac = np.full(len(masterlist_catalogue), np.nan)
ferr_app_irac[use_servs] = masterlist_catalogue['ferr_servs_irac1'][use_servs]
ferr_app_irac[use_swire] = masterlist_catalogue['ferr_swire_irac1'][use_swire]

m_app_irac = np.full(len(masterlist_catalogue), np.nan)
m_app_irac[use_servs] = masterlist_catalogue['m_servs_irac1'][use_servs]
m_app_irac[use_swire] = masterlist_catalogue['m_swire_irac1'][use_swire]

merr_app_irac = np.full(len(masterlist_catalogue), np.nan)
merr_app_irac[use_servs] = masterlist_catalogue['merr_servs_irac1'][use_servs]
merr_app_irac[use_swire] = masterlist_catalogue['merr_swire_irac1'][use_swire]

masterlist_catalogue.add_column(Column(data=f_app_irac, name="f_irac1"))
masterlist_catalogue.add_column(Column(data=ferr_app_irac, name="ferr_irac1"))
masterlist_catalogue.add_column(Column(data=m_app_irac, name="m_irac1"))
masterlist_catalogue.add_column(Column(data=merr_app_irac, name="merr_irac1"))

masterlist_catalogue.remove_columns(['f_servs_irac1', 'f_swire_irac1', 'ferr_servs_irac1',
                                     'ferr_swire_irac1', 'm_servs_irac1', 'm_swire_irac1',
                                     'merr_servs_irac1', 'merr_swire_irac1'])

origin = np.full(len(masterlist_catalogue), '     ', dtype='<U5')
origin[use_servs] = "SERVS"
origin[use_swire] = "SWIRE"
irac_origin.add_column(Column(data=origin, name="IRAC1_total"))

In [ ]:
# IRAC2 aperture flux and magnitudes
has_servs = ~np.isnan(masterlist_catalogue['f_app_servs_irac2'])
has_swire = ~np.isnan(masterlist_catalogue['f_app_swire_irac2'])
has_both = has_servs & has_swire

print("{} sources with SERVS flux".format(np.sum(has_servs)))
print("{} sources with SWIRE flux".format(np.sum(has_swire)))
print("{} sources with SERVS and SWIRE flux".format(np.sum(has_both)))

has_servs_above_limit = has_servs.copy()
has_servs_above_limit[has_servs] = masterlist_catalogue['f_app_servs_irac2'][has_servs] > 2000

use_swire = (has_swire & ~has_servs) | (has_both & has_servs_above_limit)
use_servs = (has_servs & ~(has_both & has_servs_above_limit))

print("{} sources for which we use SERVS".format(np.sum(use_servs)))
print("{} sources for which we use SWIRE".format(np.sum(use_swire)))

f_app_irac = np.full(len(masterlist_catalogue), np.nan)
f_app_irac[use_servs] = masterlist_catalogue['f_app_servs_irac2'][use_servs]
f_app_irac[use_swire] = masterlist_catalogue['f_app_swire_irac2'][use_swire]

ferr_app_irac = np.full(len(masterlist_catalogue), np.nan)
ferr_app_irac[use_servs] = masterlist_catalogue['ferr_app_servs_irac2'][use_servs]
ferr_app_irac[use_swire] = masterlist_catalogue['ferr_app_swire_irac2'][use_swire]

m_app_irac = np.full(len(masterlist_catalogue), np.nan)
m_app_irac[use_servs] = masterlist_catalogue['m_app_servs_irac2'][use_servs]
m_app_irac[use_swire] = masterlist_catalogue['m_app_swire_irac2'][use_swire]

merr_app_irac = np.full(len(masterlist_catalogue), np.nan)
merr_app_irac[use_servs] = masterlist_catalogue['merr_app_servs_irac2'][use_servs]
merr_app_irac[use_swire] = masterlist_catalogue['merr_app_swire_irac2'][use_swire]

masterlist_catalogue.add_column(Column(data=f_app_irac, name="f_app_irac2"))
masterlist_catalogue.add_column(Column(data=ferr_app_irac, name="ferr_app_irac2"))
masterlist_catalogue.add_column(Column(data=m_app_irac, name="m_app_irac2"))
masterlist_catalogue.add_column(Column(data=merr_app_irac, name="merr_app_irac2"))

masterlist_catalogue.remove_columns(['f_app_servs_irac2', 'f_app_swire_irac2', 'ferr_app_servs_irac2',
                                     'ferr_app_swire_irac2', 'm_app_servs_irac2', 'm_app_swire_irac2',
                                     'merr_app_servs_irac2', 'merr_app_swire_irac2'])

origin = np.full(len(masterlist_catalogue), '     ', dtype='<U5')
origin[use_servs] = "SERVS"
origin[use_swire] = "SWIRE"
irac_origin.add_column(Column(data=origin, name="IRAC2_app"))

In [ ]:
# IRAC2 total flux and magnitudes
has_servs = ~np.isnan(masterlist_catalogue['f_servs_irac2'])
has_swire = ~np.isnan(masterlist_catalogue['f_swire_irac2'])
has_both = has_servs & has_swire

print("{} sources with SERVS flux".format(np.sum(has_servs)))
print("{} sources with SWIRE flux".format(np.sum(has_swire)))
print("{} sources with SERVS and SWIRE flux".format(np.sum(has_both)))

has_servs_above_limit = has_servs.copy()
has_servs_above_limit[has_servs] = masterlist_catalogue['f_servs_irac2'][has_servs] > 2000

use_swire = (has_swire & ~has_servs) | (has_both & has_servs_above_limit)
use_servs = (has_servs & ~(has_both & has_servs_above_limit))

print("{} sources for which we use SERVS".format(np.sum(use_servs)))
print("{} sources for which we use SWIRE".format(np.sum(use_swire)))

f_app_irac = np.full(len(masterlist_catalogue), np.nan)
f_app_irac[use_servs] = masterlist_catalogue['f_servs_irac2'][use_servs]
f_app_irac[use_swire] = masterlist_catalogue['f_swire_irac2'][use_swire]

ferr_app_irac = np.full(len(masterlist_catalogue), np.nan)
ferr_app_irac[use_servs] = masterlist_catalogue['ferr_servs_irac2'][use_servs]
ferr_app_irac[use_swire] = masterlist_catalogue['ferr_swire_irac2'][use_swire]

m_app_irac = np.full(len(masterlist_catalogue), np.nan)
m_app_irac[use_servs] = masterlist_catalogue['m_servs_irac2'][use_servs]
m_app_irac[use_swire] = masterlist_catalogue['m_swire_irac2'][use_swire]

merr_app_irac = np.full(len(masterlist_catalogue), np.nan)
merr_app_irac[use_servs] = masterlist_catalogue['merr_servs_irac2'][use_servs]
merr_app_irac[use_swire] = masterlist_catalogue['merr_swire_irac2'][use_swire]

masterlist_catalogue.add_column(Column(data=f_app_irac, name="f_irac2"))
masterlist_catalogue.add_column(Column(data=ferr_app_irac, name="ferr_irac2"))
masterlist_catalogue.add_column(Column(data=m_app_irac, name="m_irac2"))
masterlist_catalogue.add_column(Column(data=merr_app_irac, name="merr_irac2"))

masterlist_catalogue.remove_columns(['f_servs_irac2', 'f_swire_irac2', 'ferr_servs_irac2',
                                     'ferr_swire_irac2', 'm_servs_irac2', 'm_swire_irac2',
                                     'merr_servs_irac2', 'merr_swire_irac2'])

origin = np.full(len(masterlist_catalogue), '     ', dtype='<U5')
origin[use_servs] = "SERVS"
origin[use_swire] = "SWIRE"
irac_origin.add_column(Column(data=origin, name="IRAC2_total"))

In [ ]:
irac_origin.write("data/elais-n1_irac_fluxes_origins.fits")


## X.a Wavelenght domain coverage

We add a binary `flag_optnir_obs` indicating that a source was observed in a given wavelenght domain:

- 1 for observation in optical;
- 2 for observation in near-infrared;
- 4 for observation in mid-infrared (IRAC).

It's an integer binary flag, so a source observed both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: The observation flag is based on the creation of multi-order coverage maps from the catalogues, this may not be accurate, especially on the edges of the coverage.*

*Note 2: Being on the observation coverage does not mean having fluxes in that wavelength domain. For sources observed in one domain but having no flux in it, one must take into consideration de different depths in the catalogue we are using.*

In [ ]:
was_observed_optical = utils.inMoc(
    masterlist_catalogue['ra'], masterlist_catalogue['dec'],
    wfc_moc + sparcs_moc) 

was_observed_nir = utils.inMoc(
    masterlist_catalogue['ra'], masterlist_catalogue['dec'],
    dxs_moc
)

was_observed_mir = utils.inMoc(
    masterlist_catalogue['ra'], masterlist_catalogue['dec'],
    servs_moc + swire_moc
)

In [ ]:
masterlist_catalogue.add_column(
    Column(
        1 * was_observed_optical + 2 * was_observed_nir + 4 * was_observed_mir,
        name="flag_optnir_obs")
)

## X.b Wavelength domain detection

We add a binary `flag_optnir_det` indicating that a source was detected in a given wavelenght domain:

- 1 for detection in optical;
- 2 for detection in near-infrared;
- 4 for detection in mid-infrared (IRAC).

It's an integer binary flag, so a source detected both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: We use the total flux columns to know if the source has flux, in some catalogues, we may have aperture flux and no total flux.*

To get rid of artefacts (chip edges, star flares, etc.) we consider that a source is detected in one wavelength domain when it has a flux value in **at least two bands**. That means that good sources will be excluded from this flag when they are on the coverage of only one band.

In [ ]:
# SpARCS is a catalogue of sources detected in r (with fluxes measured at 
# this prior position in the other bands).  Thus, we are only using the r
# CFHT band.
nb_optical_flux = (
    1 * ~np.isnan(masterlist_catalogue['f_wfc_u']) +
    1 * ~np.isnan(masterlist_catalogue['f_wfc_g']) +
    1 * ~np.isnan(masterlist_catalogue['f_wfc_r']) +
    1 * ~np.isnan(masterlist_catalogue['f_wfc_i']) +
    1 * ~np.isnan(masterlist_catalogue['f_wfc_z']) +
    1 * ~np.isnan(masterlist_catalogue['f_cfht_megacam_r'])
)

nb_nir_flux = (
    1 * ~np.isnan(masterlist_catalogue['f_ukidss_j']) +
    1 * ~np.isnan(masterlist_catalogue['f_ukidss_h']) +
    1 * ~np.isnan(masterlist_catalogue['f_ukidss_k'])
)

nb_mir_flux = (
    1 * ~np.isnan(masterlist_catalogue['f_irac1']) +
    1 * ~np.isnan(masterlist_catalogue['f_irac2']) +
    1 * ~np.isnan(masterlist_catalogue['f_irac3']) +
    1 * ~np.isnan(masterlist_catalogue['f_irac4'])
)

In [ ]:
has_optical_flux = nb_optical_flux >= 2
has_nir_flux = nb_nir_flux >= 2
has_mir_flux = nb_mir_flux >= 2

masterlist_catalogue.add_column(
    Column(
        1 * has_optical_flux + 2 * has_nir_flux + 4 * has_mir_flux,
        name="flag_optnir_det")
)

## X.c Summary of wavelength domains

In [ ]:
flag_obs = masterlist_catalogue['flag_optnir_obs']
flag_det = masterlist_catalogue['flag_optnir_det']

In [ ]:
venn3(
    [
        np.sum(flag_obs == 4),
        np.sum(flag_obs == 2),
        np.sum(flag_obs == 6),
        np.sum(flag_obs == 1),
        np.sum(flag_obs == 5),
        np.sum(flag_obs == 3),
        np.sum(flag_obs == 7)
    ],
    set_labels=('Optical', 'near-IR', 'mid-IR'),
    subset_label_formatter=lambda x: "{}%".format(int(100*x/len(flag_obs)))
)
plt.title("Wavelength domain observations")

In [ ]:
venn3(
    [
        np.sum(flag_det[flag_obs == 7] == 4),
        np.sum(flag_det[flag_obs == 7] == 2),
        np.sum(flag_det[flag_obs == 7] == 6),
        np.sum(flag_det[flag_obs == 7] == 1),
        np.sum(flag_det[flag_obs == 7] == 5),
        np.sum(flag_det[flag_obs == 7] == 3),
        np.sum(flag_det[flag_obs == 7] == 7)
    ],
    set_labels=('mid-IR', 'near-IR', 'Optical'),
    subset_label_formatter=lambda x: "{}%".format(int(100*x/np.sum(flag_det != 0)))
)
plt.title("Detection of the {} sources detected\n in any wavelength domains "
          "(among {} sources)".format(
              locale.format('%d', np.sum(flag_det != 0), grouping=True),
              locale.format('%d', len(flag_det), grouping=True)))

## XI - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogue. This can be used to easily get additional information from them.

In [ ]:
cross_ident_table = masterlist_catalogue[
    'help_id', 'wfc_id', 'dxs_id', 'servs_intid', 'swire_intid', 'sparcs_intid']
cross_ident_table.write("data/master_list_cross_ident_elais-n1.fits")

## XII - Cleaning and saving the master catalogue

In [ ]:
masterlist_catalogue.remove_columns([
    'wfc_id', 'dxs_id', 'servs_intid', 'swire_intid', 'sparcs_intid'])

In [ ]:
# We may want to reorder the column even if this will be done at the ingestion in HeDaM.

In [ ]:
masterlist_catalogue.write("data/master_catalogue_elais-n1.fits")